In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/filtered-llm/filtered_df_test.csv
/kaggle/input/filtered-llm/filtered_df_val.csv
/kaggle/input/filtered-llm/filtered_df_train.csv


In [41]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AdamW, get_scheduler
import torch.nn.functional as F
from tqdm import tqdm


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [43]:
train_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_train.csv')
val_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_val.csv')
test_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_test.csv')


In [44]:
X_train = train_df['xi'] + " " + train_df['xj']
X_val = val_df['xi'] + " " + val_df['xj']

y_train = train_df['LLM_model'].factorize()[0]
y_val = val_df['LLM_model'].factorize()[0]



In [45]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#bert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(y_train))).to(device)


In [46]:
def encode_data(texts, tokenizer, max_len=100):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="pt")


In [47]:
encoded_train = encode_data(X_train, tokenizer, max_len=128)
encoded_val = encode_data(X_val, tokenizer, max_len=128)


In [48]:
X_train_bert = encoded_train['input_ids']
attention_mask_train = encoded_train['attention_mask']
X_val_bert = encoded_val['input_ids']
attention_mask_val = encoded_val['attention_mask']

In [49]:
class BertDataset(Dataset):
    def __init__(self, X, attention_mask, y):
        self.X = X
        self.attention_mask = attention_mask
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.attention_mask[idx], self.y[idx]


In [50]:
train_dataset_bert = BertDataset(X_train_bert, attention_mask_train, y_train)
val_dataset_bert = BertDataset(X_val_bert, attention_mask_val, y_val)
train_loader_bert = DataLoader(train_dataset_bert, batch_size=64, shuffle=True)
val_loader_bert = DataLoader(val_dataset_bert, batch_size=64)


In [51]:
import torch.nn as nn
from transformers import BertModel, get_scheduler


In [56]:
class EnhancedModel(nn.Module):
    def __init__(self, num_classes):
        super(EnhancedModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')  
        self.dropout = nn.Dropout(p=0.2)  
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 512)  
        self.fc2 = nn.Linear(512, 256)  
        self.classifier = nn.Linear(256, num_classes)  

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        x = self.dropout(pooled_output)
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.dropout(x)
        return self.classifier(x)


In [57]:
num_classes = len(set(y_train))
bert_model = EnhancedModel(num_classes).to(device)

optimizer = AdamW(bert_model.parameters(), lr=3e-5, weight_decay=1e-4) 
num_training_steps = len(train_loader_bert) * epochs 
num_warmup_steps = int(0.1 * num_training_steps)  # 10% of total steps for warmup
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)



/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
from sklearn.metrics import roc_auc_score


In [58]:
def train_bert_model(model, train_loader, val_loader, optimizer, scheduler):
    patience = 3  
    epochs_no_improve = 0
    best_val_loss = float('inf')
    
    for param in model.bert.parameters():
        param.requires_grad = False


    for epoch in range(epochs):
        model.train()
        total_loss, total_correct = 0, 0

        for X_batch, attention_mask, y_batch in tqdm(train_loader):
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch, attention_mask=attention_mask)  # Use your modified model
            loss = F.cross_entropy(outputs, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()

        accuracy = total_correct / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}")

        # Validate
        val_loss, val_accuracy, val_auc = validate_bert_model(model, val_loader)
        print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}, AUC-ROC: {val_auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping!")
            break

# Validation loop
def validate_bert_model(model, val_loader):
    model.eval()
    total_loss, total_correct = 0, 0
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for X_batch, attention_mask, y_batch in val_loader:
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            outputs = model(X_batch, attention_mask=attention_mask)  
            loss = F.cross_entropy(outputs, y_batch)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()

            all_targets.extend(y_batch.cpu().numpy())
            all_probs.extend(F.softmax(outputs, dim=1).cpu().numpy()) 

    val_accuracy = total_correct / len(val_loader.dataset)
    val_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr')  

    return total_loss / len(val_loader), val_accuracy, val_auc

In [59]:
epochs = 10  
train_bert_model(bert_model, train_loader_bert, val_loader_bert, optimizer, lr_scheduler)


100%|██████████| 63/63 [00:16<00:00,  3.92it/s]


Epoch 1, Loss: 1.6027566308066958, Accuracy: 0.20604395604395603
Validation Loss: 1.615304633975029, Validation Accuracy: 0.2, AUC-ROC: 0.50187


100%|██████████| 63/63 [00:16<00:00,  3.76it/s]


Epoch 2, Loss: 1.5770942975604345, Accuracy: 0.27372627372627373
Validation Loss: 1.6329543441534042, Validation Accuracy: 0.186, AUC-ROC: 0.50071


100%|██████████| 63/63 [00:15<00:00,  3.94it/s]


Epoch 3, Loss: 1.5494531922870212, Accuracy: 0.3101898101898102
Validation Loss: 1.650890588760376, Validation Accuracy: 0.2, AUC-ROC: 0.50065


100%|██████████| 63/63 [00:15<00:00,  4.07it/s]


Epoch 4, Loss: 1.5265407070281014, Accuracy: 0.3194305694305694
Validation Loss: 1.6827496439218521, Validation Accuracy: 0.2, AUC-ROC: 0.502135
Early stopping!


In [60]:
def prepare_test_data(test_df, tokenizer, max_len=100):
    X_test = test_df['xi'] + " " + test_df['xj']  
    y_test = test_df['LLM_model'].factorize()[0]  

    encoded_test = encode_data(X_test, tokenizer, max_len)
    X_test_bert = encoded_test['input_ids']
    attention_mask_test = encoded_test['attention_mask']

    test_dataset_bert = BertDataset(X_test_bert, attention_mask_test, y_test)
    return DataLoader(test_dataset_bert, batch_size=64), y_test  

test_loader_bert, y_test = prepare_test_data(test_df, tokenizer, max_len=100)

def evaluate_test_data(model, test_loader):
    model.eval()
    total_loss, total_correct = 0, 0
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for X_batch, attention_mask, y_batch in test_loader:
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            outputs = model(X_batch, attention_mask=attention_mask)  # Use your modified model
            loss = F.cross_entropy(outputs, y_batch)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()
            all_targets.extend(y_batch.cpu().numpy())
            all_probs.extend(F.softmax(outputs, dim=1).cpu().numpy())

    test_accuracy = total_correct / len(test_loader.dataset)
    test_loss = total_loss / len(test_loader)
    test_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr')

    return test_loss, test_accuracy, test_auc

In [61]:
test_loss, test_accuracy, test_auc = evaluate_test_data(bert_model, test_loader_bert)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test AUC-ROC: {test_auc}")

Test Loss: 1.6798717826604843, Test Accuracy: 0.15568862275449102, Test AUC-ROC: 0.512026357373892


# OLD

In [ ]:
weight_decay = 1e-5
patience = 3
#5e-5,1e-5(23), 2e-5(25), or 3e-5
optimizer = AdamW(bert_model.parameters(), lr=3e-5, weight_decay=weight_decay)
num_training_steps = len(train_loader_bert)  # Number of steps is based on the number of batches
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


In [ ]:
def train_bert_model(model, train_loader, val_loader, optimizer, scheduler):
    global best_val_loss, epochs_no_improve  # Declare variables as global
    for epoch in range(epochs):
        model.train()
        total_loss, total_correct = 0, 0
        for X_batch, attention_mask, y_batch in tqdm(train_loader):
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch, attention_mask=attention_mask).logits
            loss = F.cross_entropy(outputs, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()

        accuracy = total_correct / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}")

        # Validate
        val_loss, val_accuracy = validate_bert_model(model, val_loader)
        print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping!")
                break


In [ ]:
best_val_loss = float('inf')
epochs_no_improve = 0
epochs = 10  # Define your number of epochs if needed
train_bert_model(bert_model, train_loader_bert, val_loader_bert, optimizer, lr_scheduler)


In [ ]:
# Hyperparameter tuning parameters
learning_rates = [1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [16, 32, 64]
weight_decays = [0.0, 0.01, 0.1]  # Tuning weight decay
warmup_steps = [0, 100, 200]  # Tuning warm-up steps
num_epochs = 5 

def hyperparameter_tuning(learning_rates, batch_sizes, weight_decays, warmup_steps):
    best_accuracy = 0
    best_params = {}

    for lr, batch_size, weight_decay, warmup in itertools.product(learning_rates, batch_sizes, weight_decays, warmup_steps):
        print(f"Training with lr={lr}, batch_size={batch_size}, weight_decay={weight_decay}, warmup_steps={warmup}")

        train_dataset_bert = BertDataset(X_train_bert, attention_mask_train, y_train)
        val_dataset_bert = BertDataset(X_val_bert, attention_mask_val, y_val)
        train_loader_bert = DataLoader(train_dataset_bert, batch_size=batch_size, shuffle=True)
        val_loader_bert = DataLoader(val_dataset_bert, batch_size=batch_size)

        optimizer = AdamW(bert_model.parameters(), lr=lr, weight_decay=weight_decay)

        num_training_steps = num_epochs * len(train_loader_bert)
        num_warmup_steps = warmup if warmup > 0 else int(0.1 * num_training_steps)  

        lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

        def train_bert_model(model, train_loader, val_loader, optimizer, scheduler, epochs):
            for epoch in range(epochs):
                model.train()
                total_loss, total_correct = 0, 0
                for X_batch, attention_mask, y_batch in tqdm(train_loader):
                    X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    outputs = model(X_batch, attention_mask=attention_mask).logits
                    loss = F.cross_entropy(outputs, y_batch)
                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                    total_loss += loss.item()
                    total_correct += (outputs.argmax(1) == y_batch).sum().item()

                accuracy = total_correct / len(train_loader.dataset)
                print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Accuracy: {accuracy}")

                # Validate
                val_loss, val_accuracy = validate_bert_model(model, val_loader)
                print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        def validate_bert_model(model, val_loader):
            model.eval()
            total_loss, total_correct = 0, 0
            with torch.no_grad():
                for X_batch, attention_mask, y_batch in val_loader:
                    X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
                    outputs = model(X_batch, attention_mask=attention_mask).logits
                    loss = F.cross_entropy(outputs, y_batch)
                    total_loss += loss.item()
                    total_correct += (outputs.argmax(1) == y_batch).sum().item()
            
            val_accuracy = total_correct / len(val_loader.dataset)
            return total_loss / len(val_loader), val_accuracy

        train_bert_model(bert_model, train_loader_bert, val_loader_bert, optimizer, lr_scheduler, num_epochs)

In [ ]:
hyperparameter_tuning(learning_rates, batch_sizes, weight_decays, warmup_steps)
